In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
spark.sql("use twitter")

DataFrame[]

In [4]:
df = spark.sql("select * from twit")

In [5]:
df

DataFrame[tw_date: string, id: int, url: string, username: string, source: string, location: string, tweet: string, num_of_likes: int, num_of_retweet: int]

In [6]:
df.printSchema()

root
 |-- tw_date: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- url: string (nullable = true)
 |-- username: string (nullable = true)
 |-- source: string (nullable = true)
 |-- location: string (nullable = true)
 |-- tweet: string (nullable = true)
 |-- num_of_likes: integer (nullable = true)
 |-- num_of_retweet: integer (nullable = true)



In [7]:
spark.sql("select * from twit")

DataFrame[tw_date: string, id: int, url: string, username: string, source: string, location: string, tweet: string, num_of_likes: int, num_of_retweet: int]

In [8]:
df.show()

+--------------------+----+--------------------+---------------+-------------------+------------------+--------------------+------------+--------------+
|             tw_date|  id|                 url|       username|             source|          location|               tweet|num_of_likes|num_of_retweet|
+--------------------+----+--------------------+---------------+-------------------+------------------+--------------------+------------+--------------+
|                Date|null|                 url|       username|             source|          location|               tweet|        null|          null|
|2023-02-22 23:33:...|null|https://twitter.c...|    Anupbhatia7|Twitter for Android|           Unknown|#bjpIsittrueourho...|          26|             7|
|2023-02-22 23:08:...|null|https://twitter.c...|   Dr__Antidote|Twitter for Android|             🇮🇳 |Indian#PrimeMinis...|          14|             5|
|2023-02-22 23:07:...|null|https://twitter.c...|    Asifwani786|Twitter for Android|

In [9]:
type(df)

pyspark.sql.dataframe.DataFrame

In [10]:
df.groupBy("location").count().show()

+--------------------+-----+
|            location|count|
+--------------------+-----+
|           Bangalore|    4|
|   Lost in the crowd|    3|
|         "Vijayawada|    1|
|             Raichur|    1|
|             "Barbil|    1|
|       Kallakurichi |    1|
|    "Bengaluru South|    2|
|           "Nainital|    1|
|           " Athgarh|    1|
|Urbs Prima In Indis!|    1|
|   QUAZIGUND KASHMIR|    1|
|                "6-A|    1|
|         Blue planet|    1|
|     "Ministers Hill|    1|
|        "தொண்டை நாடு|    1|
|             "Manila|    1|
|          "Hindustan|    1|
|    Santha Baraabara|    1|
|                भारत|    1|
|              Sweden|    1|
+--------------------+-----+
only showing top 20 rows



In [17]:
# Find the tweet with the highest number of likes:
df1=spark.sql("SELECT * FROM twit ORDER BY num_of_likes DESC LIMIT 1")
df1.show()

+--------------------+----+--------------------+------------+------------------+--------+--------+------------+--------------+
|             tw_date|  id|                 url|    username|            source|location|   tweet|num_of_likes|num_of_retweet|
+--------------------+----+--------------------+------------+------------------+--------+--------+------------+--------------+
|2023-02-22 03:42:...|null|https://twitter.c...|narendramodi|Twitter for iPhone|   India|Verygood|       30257|          4890|
+--------------------+----+--------------------+------------+------------------+--------+--------+------------+--------------+



In [18]:
# Find the user with the most number of tweets in this dataset:
d3=spark.sql("SELECT username, COUNT(*) AS tweet_count FROM twit GROUP BY username ORDER BY tweet_count DESC LIMIT 1")
d3.show()

+--------------+-----------+
|      username|tweet_count|
+--------------+-----------+
|TanyaGhosh2312|         14|
+--------------+-----------+



In [19]:
# Find the top 5 tweets with the most number of retweets:
d4=spark.sql("SELECT * FROM twit ORDER BY num_of_retweet DESC LIMIT 5")
d4.show()

+--------------------+----+--------------------+---------------+-------------------+-------------+--------------------+------------+--------------+
|             tw_date|  id|                 url|       username|             source|     location|               tweet|num_of_likes|num_of_retweet|
+--------------------+----+--------------------+---------------+-------------------+-------------+--------------------+------------+--------------+
|2023-02-22 01:59:...|null|https://twitter.c...|     singhvarun| Twitter for iPhone|      "Mumbai|              India"|        null|          5430|
|2023-02-22 03:42:...|null|https://twitter.c...|   narendramodi| Twitter for iPhone|        India|            Verygood|       30257|          4890|
|2023-02-22 07:21:...|null|https://twitter.c...|theskindoctor13|Twitter for Android|"Flower Mound|                 TX"|        null|          4700|
|2023-02-22 18:05:...|null|https://twitter.c...|    VarunKrRana|Twitter for Android|   "New Delhi|              

In [20]:
# Find the average number of likes and retweets for all tweets in the dataset:
d5=spark.sql("SELECT AVG(num_of_likes) AS avg_likes, AVG(num_of_retweet) AS avg_retweets FROM twit")
d5.show()

+-----------------+-----------------+
|        avg_likes|     avg_retweets|
+-----------------+-----------------+
|276.5952023988006|98.66869918699187|
+-----------------+-----------------+



In [21]:
# Find the number of tweets posted from each source (e.g., Twitter for Android, Twitter for iPhone):
d6=spark.sql("SELECT source, COUNT(*) AS tweet_count FROM twit GROUP BY source")
d6.show()

+--------------------+-----------+
|              source|tweet_count|
+--------------------+-----------+
|             Emplifi|          2|
|ProxyCentral CNNn...|          2|
|     Twitter Web App|        161|
|Twitter Media Studio|         18|
|              source|          1|
|  Twitter for iPhone|        242|
|              Buffer|          1|
|    Twitter for iPad|          7|
|           TweetDeck|         22|
|          SocialFlow|          1|
|          Desk Tweet|          1|
|              Grabyo|          5|
| Twitter for Android|        533|
|             Echobox|          1|
|            Hypefury|          1|
|        Nanda8Tooter|          1|
|      Hootsuite Inc.|          2|
+--------------------+-----------+



In [22]:
# Find the tweets posted by a specific user (e.g., Anupbhatia7):
d7=spark.sql("SELECT * FROM twit WHERE username = 'Anupbhatia7'")
d7.show()

+--------------------+----+--------------------+-----------+-------------------+--------+--------------------+------------+--------------+
|             tw_date|  id|                 url|   username|             source|location|               tweet|num_of_likes|num_of_retweet|
+--------------------+----+--------------------+-----------+-------------------+--------+--------------------+------------+--------------+
|2023-02-22 23:33:...|null|https://twitter.c...|Anupbhatia7|Twitter for Android| Unknown|#bjpIsittrueourho...|          26|             7|
+--------------------+----+--------------------+-----------+-------------------+--------+--------------------+------------+--------------+



In [24]:
# Find the tweets posted on a specific date (e.g., February 22, 2023):
d8=spark.sql("SELECT * FROM twit WHERE DATE_TRUNC('day', tw_date) = '2023-02-22'")
d8.show()

+--------------------+----+--------------------+---------------+-------------------+--------------------+--------------------+------------+--------------+
|             tw_date|  id|                 url|       username|             source|            location|               tweet|num_of_likes|num_of_retweet|
+--------------------+----+--------------------+---------------+-------------------+--------------------+--------------------+------------+--------------+
|2023-02-22 18:27:...|null|https://twitter.c...|     vijeshetty| Twitter for iPhone|          "Bengaluru|              India"|        null|            45|
|2023-02-22 18:16:...|null|https://twitter.c...|     sreebalann| Twitter for iPhone|             Unknown|GeorgeSOROSwasbeh...|          11|             5|
|2023-02-22 18:14:...|null|https://twitter.c...|     PC67761415|Twitter for Android|             Unknown|@QamarcheemaWatch...|          25|            25|
|2023-02-22 18:12:...|null|https://twitter.c...|   HardeepSPuri| Twitt